In [ ]:
import cv2
import os
import numpy as np
from tqdm import tqdm
from PIL import Image

# Merge Image

In [ ]:
def merge_images(input_dir, output_dir):
    files = os.listdir(input_dir)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for file in tqdm(files):
        if file.startswith("predictleft_") and file.endswith(".jpg"):
            left_image_path = os.path.join(input_dir, file)
            right_image_id = file.removeprefix("predictleft_").removesuffix(".jpg")
            right_image_path = os.path.join(input_dir, f"predictright_{right_image_id}.jpg")
            if os.path.exists(right_image_path):
                left_image = Image.open(left_image_path)
                right_image = Image.open(right_image_path)
                right_image = right_image.transpose(Image.FLIP_LEFT_RIGHT)
                merged_image = Image.new("1", (640, 320))
                merged_image.paste(left_image, (0, 0))
                merged_image.paste(right_image, (320, 0))
                output_path = os.path.join(output_dir, f"{right_image_id}.png")
                merged_image.save(output_path)
            else:
                print(f"Right Image Not Found: {right_image_path}")

In [ ]:
# log = "mydataset_2023_12_26_15_48_14"
# merge_images(f"./logs/{log}/Samples", f"./logs/{log}/Samples_merged")

In [ ]:
def merge_images_overlap(input_dir, output_dir):
    files = os.listdir(input_dir)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for file in tqdm(files):
        if file.startswith("predictleft_") and file.endswith(".jpg"):
            left_image_path = os.path.join(input_dir, file)
            right_image_id = file.removeprefix("predictleft_").removesuffix(".jpg")
            right_image_path = os.path.join(input_dir, f"predictright_{right_image_id}.jpg")
            if os.path.exists(right_image_path):
                left_image = Image.open(left_image_path)
                right_image = Image.open(right_image_path)
                right_image = right_image.transpose(Image.FLIP_LEFT_RIGHT)
                left_temp = Image.new("1", (640, 320))
                right_temp = Image.new("1", (640, 320))
                left_temp.paste(left_image, (30, 0))
                right_temp.paste(right_image, (290, 0))
                merged_image = ImageChops.logical_or(left_temp, right_temp)
                output_path = os.path.join(output_dir, f"{right_image_id}.png")
                merged_image.save(output_path)
            else:
                print(f"Right Image Not Found: {right_image_path}")

In [ ]:
# log = "mydataset_2023_12_26_15_48_14"
# merge_images_overlap(f"./logs/{log}/Samples", f"./logs/{log}/Samples_merged")

# Filter Small Components

In [ ]:
def filter_small_connected_components(binary_image, threshold):
    nb_components, output, stats, _ = cv2.connectedComponentsWithStats(binary_image, connectivity=8)
    sizes = stats[:, -1]
    filtered_image = np.zeros(output.shape)
    for i in range(1, nb_components):
        if sizes[i] >= threshold:
            filtered_image[output == i] = 255
    return filtered_image



def process_images_in_directory(input_dir, output_dir, threshold):
    os.makedirs(output_dir, exist_ok=True)
    for filename in tqdm(os.listdir(input_dir)):
        if filename.endswith((".png", ".jpg", ".jpeg")):
            image_path = os.path.join(input_dir, filename)
            binary_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            filtered_image = filter_small_connected_components(binary_image, threshold).astype(np.bool_)
            pil_image = Image.fromarray(filtered_image)
            output_path = os.path.join(output_dir, filename)
            pil_image.save(output_path)

In [ ]:
log = "mydataset_2023_12_26_12_47_36"
process_images_in_directory(f"./logs/{log}/Samples_merged", f"./logs/{log}/Samples_merged_filtered", 200)

# Convert To Binary Image

In [ ]:
def convert_to_binary(image_path, threshold=128):
    img = Image.open(image_path)
    gray_img = img.convert('L')
    binary_img = gray_img.point(lambda pixel: 0 if pixel < threshold else 255, "1")
    return binary_img


def batch_convert_to_binary(input_directory, output_directory, threshold=128):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for filename in os.listdir(input_directory):
        if filename.endswith(".jpg"):
            input_path = os.path.join(input_directory, filename)
            output_path = os.path.join(output_directory, filename.replace("predict", "").replace(".jpg", ".png"))
            binary_image = convert_to_binary(input_path, threshold)
            binary_image.save(output_path)

In [ ]:
# log = "mydataset_2023_12_26_15_48_14"
# batch_convert_to_binary(f"./logs/{log}/Samples_merged", f"./logs/{log}/Samples_merged_binary", 128)